In [ ]:
# Import Libraries
import imageio.v2 as imageio
import matplotlib.pyplot as plt
import scipy.ndimage as ndi
import numpy as np
import pandas as pd
from scipy.stats import ttest_ind

# Load the Ct Scan Data Directory
liverCtScansDir = imageio.imread("100.dcm")
liverCtScansDirVol = imageio.volread("23-2", format='dicom')
datavol = imageio.volread("dataset", format='dicom')


# patient and image info
print('Available Metadata For Liver Ct Scans:', liverCtScansDirVol.meta.keys())
print('Shape of Each Liver Ct Image Array:', liverCtScansDirVol.shape)
print('Patient Sex: ', liverCtScansDirVol.meta['PatientSex'])
print('Patient Birthdate: ', liverCtScansDirVol.meta['PatientBirthDate'])
print('Pixel Spacing: ', liverCtScansDirVol.meta['PixelSpacing'])
print('Rescale Slope: ', liverCtScansDirVol.meta['RescaleSlope'])


# Sampling, Pixel Aspect Ratio, Field of View
datavol_n0, datavol_n1, datavol_n2 = datavol.shape
print("Number of Slices:\n\t", "Axial=", datavol_n0, "Slices\n\t", "Coronal=", datavol_n1, "Slices\n\t", "Sagittal=", datavol_n2, "Slices")


datavol_d0, datavol_d1, datavol_d2 = datavol.meta['sampling']
print("Sampling:\n\t", "Axial=", datavol_d0, "mm\n\t", "Coronal=", datavol_d1, "mm\n\t", "Sagittal=", datavol_d2, "mm")


datavol_axialAspect = datavol_d1 / datavol_d2
datavol_sagittalAspect = datavol_d0 / datavol_d1
datavol_coronalAspect = datavol_d0 / datavol_d2
print("Pixel Aspect Ratio:\n\t", "Axial=", datavol_axialAspect, "\n\t", "Coronal=", datavol_coronalAspect, "\n\t", "Sagittal=", datavol_sagittalAspect)


print("Field of View:\n\t", "Axial=", datavol_n0*datavol_d0, "mm\n\t", "Coronal=", datavol_n1*datavol_d1, "mm\n\t", "Sagittal=", datavol_n2*datavol_d2, "mm")


# Slice CT Image from Different Plates
liverCTSlice_figure, liverCTSlice_axes = plt.subplots(nrows=1, ncols=4)
for ctDataDirVolIndex in range(4):
  liverCtDataDirVolSlice = liverCtScansDirVol[ctDataDirVolIndex * 10, :, :]
  liverCTSlice_axes[ctDataDirVolIndex].imshow(liverCtDataDirVolSlice, cmap='gray')
  liverCTSlice_axes[ctDataDirVolIndex].axis('off')

plt.suptitle("Liver Axial")
plt.show()


liverCTSlice_figure, dataCTSlice_axes = plt.subplots(nrows=1, ncols=4)
for ctDataDirVolIndex in range(4):
  datavolSlice = datavol[ctDataDirVolIndex * 10, :, :]
  dataCTSlice_axes[ctDataDirVolIndex].imshow(datavolSlice, cmap='gray')
  dataCTSlice_axes[ctDataDirVolIndex].axis('off')

plt.suptitle("Axial")
plt.show()


datavol_firstSlice = datavol[:, 40, :]
datavol_secondSlice = datavol[:, :, 40]


liverCTSlice_figure, liverCTSlice_ax = plt.subplots(1, 3)
# Axial Plane: show the 10th slice
liverCTSlice_ax[0].imshow(datavol[10, :, :], cmap='gray', aspect= datavol_axialAspect)
liverCTSlice_ax[0].set_title('Axial')

# Coronal Plane: show the slice 40
liverCTSlice_ax[1].imshow(datavol_firstSlice, cmap='gray', aspect= datavol_coronalAspect)
liverCTSlice_ax[1].set_title('Coronal')

# Sagittal Plane: show the slice 40
liverCTSlice_ax[2].imshow(datavol_secondSlice, cmap='gray', aspect= datavol_sagittalAspect)
#liverCTSlice_ax[2].axis('off')
liverCTSlice_ax[2].set_title('Sagittal')
plt.show()

# Morphology and Histogram Enhance For CT Scans by Applying Appropriate Morphological Operations

liverCTSlice_figure, liverCTSlice_axes = plt.subplots(nrows=1, ncols=4)
for i in range(4):
  liverCTSlice_histogram = ndi.histogram(liverCtScansDirVol[i * 10, :, :], min=0, max=255, bins=256)
  liverCTSlice_axes[i].plot(liverCTSlice_histogram)

plt.suptitle("Before Histogram")
plt.show()

#plotting of masks
liverCtScans_mask1 = liverCtScansDir > 50
liverCtScans_mask2 = liverCtScansDir > 100
liverCtScans_mask3 = liverCtScans_mask1 + ~liverCtScans_mask2

liverCTSlice_figure, liverCTSlice_axes = plt.subplots(1, 3)

liverCTSlice_axes[0].imshow(liverCtScans_mask1 * 255)
liverCTSlice_axes[1].imshow(liverCtScans_mask2 * 255)
liverCTSlice_axes[2].imshow(liverCtScans_mask3 * 255)

liverCTSlice_axes[0].set_title('mask > 50')
liverCTSlice_axes[1].set_title('mask > 100')
liverCTSlice_axes[2].set_title('m1 + ~ m2')

for i in range(3):
    liverCTSlice_axes[i].axis('off')
plt.suptitle("Masks")
plt.show()


liverCtScans_imgMask1 = np.where(liverCtScans_mask1, liverCtScansDir, 0)
liverCtScans_imgMask2 = np.where(liverCtScans_mask2, liverCtScansDir, 0)
liverCtScans_imgMask3 = np.where(liverCtScans_mask3, liverCtScansDir, 0)



#plotting of image after applying masks
liverCTSlice_figure, liverCTSlice_axes = plt.subplots(1, 3)

liverCTSlice_axes[0].imshow(liverCtScans_imgMask1 * 255)
liverCTSlice_axes[1].imshow(liverCtScans_imgMask2 * 255)
liverCTSlice_axes[2].imshow(liverCtScans_imgMask3 * 255)

for i in range(3):
    liverCTSlice_axes[i].axis('off')
plt.suptitle("Images with Masks")
plt.show()


liverCTSlice_figure, liverCTSlice_axes = plt.subplots(nrows=1, ncols=3)
for i in range(3):

  liverCTSlice_histogram1 = ndi.histogram(liverCtScans_imgMask1, min=0, max=255, bins=256)
  liverCTSlice_axes[i].plot(liverCTSlice_histogram1)
  liverCTSlice_histogram2 = ndi.histogram(liverCtScans_imgMask2, min=0, max=255, bins=256)
  liverCTSlice_axes[i].plot(liverCTSlice_histogram2)
  liverCTSlice_histogram3 = ndi.histogram(liverCtScans_imgMask3, min=0, max=255, bins=256)
  liverCTSlice_axes[i].plot(liverCTSlice_histogram3)

plt.suptitle("After Histogram")
plt.show()


#filter and liverCTSlice_mask for feature detection
#sharpening
liverCTSlice_weights = [[0, -1, 0], [-1, 5, -1], [0, -1, 0]]
liverCTSlice_imgFilter = ndi.convolve(liverCtScansDir, liverCTSlice_weights)

liverCTSlice_figure, liverCTSlice_axes = plt.subplots(1, 2)
liverCTSlice_axes[0].imshow(liverCtScansDir)
liverCTSlice_axes[0].set_title('og')
liverCTSlice_axes[1].imshow(liverCTSlice_imgFilter)
liverCTSlice_axes[1].set_title('filtered')
plt.suptitle("Sharpning")
plt.show()


#smooth
liverCTSlice_img_s1 = ndi.gaussian_filter(liverCtScansDir, sigma=1)
liverCTSlice_img_s3 = ndi.gaussian_filter(liverCtScansDir, sigma=3)

liverCTSlice_figure, liverCTSlice_axes = plt.subplots(1, 3)
liverCTSlice_axes[0].imshow(liverCtScansDir)
liverCTSlice_axes[0].set_title('og')
liverCTSlice_axes[1].imshow(liverCTSlice_img_s1)
liverCTSlice_axes[1].set_title('filtered s=1')
liverCTSlice_axes[2].imshow(liverCTSlice_img_s3)
liverCTSlice_axes[2].set_title('filtered s=3')
plt.suptitle("Smoothing")
plt.show()


# Feature Detection
liverCTSlice_weights = [[+1, 0, -1], [+1, 0, -1], [+1, 0, -1]]
liverCTSlice_edges = ndi.convolve(liverCtScansDir, liverCTSlice_weights)
plt.imshow(liverCTSlice_edges, vmin= 0, vmax=150)
plt.suptitle("Feature Detection")
plt.show()



#edge detection
liverCTSlice_sobel_ax0 = ndi.sobel(liverCtScansDir, axis=0)
liverCTSlice_sobel_ax1 = ndi.sobel(liverCtScansDir, axis=1)
liverCTSlice_gradient_magnitude = np.square(liverCTSlice_sobel_ax0) + np.square(liverCTSlice_sobel_ax1)
liverCTSlice_gradient = liverCTSlice_gradient_magnitude * 1/2 * 255
plt.imshow(liverCTSlice_gradient, cmap='gray', vmax=75)
plt.show()


# Object Extraction and Segmentation
liverCTSlice_imgFilter = ndi.median_filter(liverCtScansDir, size=3)
liverCTSlice_maskStart = np.where(liverCTSlice_imgFilter > 60, 1, 0)
liverCTSlice_mask = ndi.binary_closing(liverCTSlice_maskStart)
liverCTSlice_labels, liverCTSlice_nlabels = ndi.label(liverCTSlice_mask)
print('Num. Labels:', liverCTSlice_nlabels)
overlay = np.where(liverCTSlice_labels > 0, liverCTSlice_labels,
                   np.nan)
plt.imshow(overlay, cmap='rainbow')
plt.axis('off')
plt.show()


liverCTSlice_lv_mask = np.where(liverCTSlice_labels == 0, liverCtScansDir, np.nan)
plt.imshow(liverCTSlice_lv_mask)
plt.axis('off')
plt.show()

liverCTSlice_lv_box = ndi.find_objects(liverCTSlice_labels == 0)
print("obj: ",len(liverCTSlice_lv_box))
print(liverCTSlice_lv_box[0])
im_lv = liverCtDataDirVolSlice[liverCTSlice_lv_box[0]]
plt.imshow(im_lv)
plt.axis('off')
plt.show()

print(ndi.mean(liverCtScansDir))
print(ndi.mean(liverCtScansDir, liverCTSlice_labels))
print(ndi.mean(liverCtScansDir, liverCTSlice_labels, index=0))
print(ndi.mean(liverCtScansDir, liverCTSlice_labels, index=[1, 2]))

# Morophology Operation and Measuring In Time
liverCTSlice_img_fit = ndi.median_filter(liverCtScansDir, size=3)

# Masking
liverCTSlice_maskStart = np.where(liverCTSlice_img_fit > 60, 1, 0)
liverCTSlice_mask = ndi.binary_closing(liverCTSlice_maskStart)

# Labeling
liverCTSlice_labels, liverCTSlice_nlabels = ndi.label(liverCTSlice_mask)

# Calculate liverCTSlice_area of LV
liverCtDataDirVol_d1, liverCtDataDirVol_d2 = liverCtScansDir.meta['sampling']
liverCTSlice_dpixel = liverCtDataDirVol_d1 * liverCtDataDirVol_d2
liverCTSlice_npixels = ndi.sum(1, liverCTSlice_labels, index=0)
liverCTSlice_area = liverCTSlice_dpixel * liverCTSlice_npixels
print("Calculated Area: ", liverCTSlice_area)

# Calculate lt distances to BG
liverCTSlice_lv_label=np.where(liverCTSlice_labels == 0, 1, 0)
liverCTSlice_lv_dist = ndi.distance_transform_edt(liverCTSlice_lv_label, sampling=liverCtScansDir.meta['sampling'])
print("Max Distance: ", ndi.maximum(liverCTSlice_lv_dist))
print("Max Location: ", ndi.maximum_position(liverCTSlice_lv_dist))

# Calculate Center of Mass
liverCTSlice_com = ndi.center_of_mass(liverCtScansDir, liverCTSlice_labels, index=0)
print("Center of Mass: ", liverCTSlice_com)

# Size of Object, Distance Transformation, Centre of Mass
liverCTSlice_h, liverCTSlice_w = liverCtScansDir.shape
liverCTSlice_com = ndi.center_of_mass(liverCtScansDir)

# Calculate amount of shift needed
liverCtDataDirVol_d0 = (liverCTSlice_h / 2) - liverCTSlice_com[0]
liverCtDataDirVol_d1 = (liverCTSlice_w / 2) - liverCTSlice_com[1]

# Translate the brain towards the center
xfm = ndi.shift(liverCtScansDir, shift=(liverCtDataDirVol_d0, liverCtDataDirVol_d1))
xfmR = ndi.rotate(xfm, angle=20, reshape=False)
plt.imshow(xfmR, cmap='gray', vmax=75)
plt.show()

#Translate and Rescale
liverCTSlice_matrix = [[0.9, 0, 50]
       , [0, 0.9,-100]
       , [0, 0, 1]]
liverCTSlice_trans = ndi.affine_transform(liverCtScansDir, liverCTSlice_matrix)
plt.imshow(liverCTSlice_trans)
plt.show()


# Resample image
liverCTSlice_img_dn = ndi.zoom(xfmR, zoom=0.1)
liverCTSlice_im_up = ndi.zoom(xfmR, zoom=6)
print(liverCtScansDir.shape)
print(liverCTSlice_img_dn.shape)
print(liverCTSlice_im_up.shape)

# Resample image
liverCTSlice_up0 = ndi.zoom(liverCtScansDir, zoom=10, order=0)
liverCTSlice_up5 = ndi.zoom(liverCtScansDir, zoom=10, order=4)

# Print original and new shape
print('Original Shape: ', liverCtScansDir.shape)
print('Upsampled Shape: ', liverCTSlice_up0.shape)





liverCtDataDirVol_secondSlice = imageio.imread("101.dcm")
liverCTSlice_im3 = imageio.imread("102.dcm")

# Absu Error
liverCTSlice_absDiff = np.abs(liverCtScansDir - liverCtDataDirVol_secondSlice)
plt.imshow(liverCTSlice_absDiff, cmap='seismic', vmin=200, vmax=200)
plt.show()

liverCTSlice_error = np.mean(np.abs(liverCtScansDir - liverCtDataDirVol_secondSlice))
print("mean: ", liverCTSlice_error)
liverCTSlice_abs_dif = np.abs(liverCtScansDir - liverCTSlice_im3)
plt.imshow(liverCTSlice_abs_dif, cmap='seismic', vmin=200, vmax=200)
plt.axis('off')
plt.show()

liverCTSlice_err = np.mean(np.abs(liverCtScansDir - liverCTSlice_im3))

print("Mean: ", liverCTSlice_err)

# Measring Stastic In Ct Scans Data

liverCTSlice_df = pd.read_csv("liver_volumes.csv")
print("Sample: ", liverCTSlice_df.sample(3))

liverCTSlice_male = liverCTSlice_df.loc[liverCTSlice_df.sex == 'M', 'liver_vol']
liverCTSlice_female = liverCTSlice_df.loc[liverCTSlice_df.sex == 'F', 'liver_vol']
liverCTSlice_res = ttest_ind(liverCTSlice_male, liverCTSlice_female)
print('T = ', liverCTSlice_res.statistic)
print('P = ', liverCTSlice_res.pvalue)

liverCTSlice_df['tumor_vol'] = liverCTSlice_df.liver_vol / liverCTSlice_df.size
liverCTSlice_m_c = liverCTSlice_df.loc[liverCTSlice_df.sex == 'M', 'tumor_vol']
liverCTSlice_F_r = liverCTSlice_df.loc[liverCTSlice_df.sex == 'F', 'tumor_vol']
liverCTSlice_result = ttest_ind(liverCTSlice_m_c, liverCTSlice_F_r)
print('T = ', liverCTSlice_result.statistic)
print('P = ', liverCTSlice_result.pvalue)